# スタッキング002

In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering import FeatureEngineering
from models import Model1Xgb, Model1NN, Model2Linear

%matplotlib inline
gc.enable()

print("OK\n")

OK



In [3]:
### LightGBM
params_lgb = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
### Xgboost
params_xgb = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
}

In [4]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [5]:
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [6]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成1

In [7]:
### application
df = FeatureEngineering2.application()
df.head()

"Application_Train_Test" final shape: (356250, 234)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,-167.498254,-88.782372,1.733333,202500.0,-317.896389,1.158397,-7825.5,-317.896389,1.780220,8824.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,-125.080219,-164.152505,2.000000,67500.0,-300.000000,1.000000,-1125.0,-300.000000,3.622222,18821.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,-980.478916,NaN,4.222222,121500.0,-39.993417,1.000000,-11740.5,-39.993417,0.364055,16894.0


In [8]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

"Bureau/Bureau Balance" final shape: (305811, 200)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,135000.0,85245.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,22248.0,112500.0,69133.5,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,94500.0,94537.8,94518.9,0.0,0.0,0.0,0.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,146250.0,146250.0,146250.0,0.0,0.0,0.0,0.0


In [9]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

"Previous Applications" final shape: (338857, 321)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_PREV_GOODS_DIFF_MEAN,REFUSED_PREV_GOODS_DIFF_MAX,REFUSED_PREV_GOODS_DIFF_SUM,REFUSED_PREV_GOODS_APPL_RATIO_MEAN,REFUSED_PREV_GOODS_APPL_RATIO_MAX,REFUSED_DAYS_LAST_DUE_DIFF_MEAN,REFUSED_DAYS_LAST_DUE_DIFF_MAX,REFUSED_DAYS_LAST_DUE_DIFF_SUM,REFUSED_SIMPLE_INTERESTS_MEAN,REFUSED_SIMPLE_INTERESTS_MAX
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.015231,0.015231
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

"Pos-Cash" balance final shape: (337252, 46)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT,POS_LOAN_COMPLETED_MEAN,POS_COMPLETED_BEFORE_MEAN,POS_REMAINING_INSTALMENTS,POS_REMAINING_INSTALMENTS_RATIO,LATE_PAYMENT_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.000000,0.0,0.0,19.0,0.000000,0.0,6.0,0.250000,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,0.0,0.0,28.0,0.250000,1.0,1.0,0.083333,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,0.0,0.0,4.0,0.250000,1.0,0.0,0.000000,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.047619,0.000000,0.0,0.0,21.0,0.458333,2.0,3.0,0.250000,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.015152,0.0,0.0,66.0,0.209402,1.0,14.0,0.641667,0.0


In [11]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

"Installments Payments" final shape: (339587, 85)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1.0,-17.363636,-24.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,1.0,-6.181818,-23.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1.0,-2.250000,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

"Credit Card Balance" final shape: (103558, 284)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1168 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1122 features are remained after removing non-informative features
123 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 999 features are remained after removing features not interesting for LightGBM classifier
---=> 58 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1057 features

---=> df final shape: (356250, 1057)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,0,0,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,0,0,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,0,0,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,1,1,0,0,0,1,0,1,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,0,0,0,0,0,0,0


In [14]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1057 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1051 features are remained after removing non-informative features
40 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 1011 features are remained after removing features not interesting for LightGBM classifier
---=> 19 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1030 features

---=> df final shape: (356250, 1030)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


In [15]:
# infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


In [16]:
### 変数名から不正な文字を除去
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


In [17]:
# split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 1030)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,0,1,1,0,0,0,0,0
307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,0,0,1,1,0,0,0,0,0
307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0,0,1,1,0,1,0,1,0
307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0,1,1,0,1,0,1,0
307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [18]:
pred_train_1b = pd.read_csv('./home-credit-default-risk/exports/pred_train_1b.csv')
pred_test_1b =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1b.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.042863
1,100005,0.140563
2,100013,0.025033
3,100028,0.032262
4,100038,0.205365


In [19]:
app_train = app_train.merge(pred_train_1b, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_1b, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,1,1,0,0,0,0,0,0.042863
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,1,1,0,0,0,0,0,0.140563
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0,1,1,0,1,0,1,0,0.025033
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,0,1,1,0,1,0,1,0,0.032262
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,1,0,0,0,0,0,0.205365


In [20]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [21]:
app_train = app_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.042863,0.040068
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,1,0,0,0,0,0,0.140563,0.095240
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,1,0,1,0,0.025033,0.044435
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,1,0,1,0,0.032262,0.036586
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.205365,0.141754


In [22]:
train_set3a = pd.read_csv('./home-credit-default-risk/exports/train_set3a.csv')
test_set3a =  pd.read_csv('./home-credit-default-risk/exports/test_set3a.csv')
test_set3a.head()

,SK_ID_CURR,set3a
0,100001,0.049120
1,100005,0.115729
2,100013,0.064166
3,100028,0.038931
4,100038,0.195575


In [23]:
app_train = app_train.merge(train_set3a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,1,0,0,0,0,0,0.042863,0.040068,0.049120
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,1,0,0,0,0,0,0.140563,0.095240,0.115729
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,1,0,1,0,1,0,0.025033,0.044435,0.064166
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,1,0,1,0,1,0,0.032262,0.036586,0.038931
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,1,0,0,0,0,0,0.205365,0.141754,0.195575


In [24]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,0,0,100001,NaN,1,0,0,0,135000.0,568800.0,...,1,1,0,0,0,0,0,0.042863,0.040068,0.049120
1,1,1,100005,NaN,0,0,0,0,99000.0,222768.0,...,1,1,0,0,0,0,0,0.140563,0.095240,0.115729
2,2,2,100013,NaN,0,1,0,0,202500.0,663264.0,...,1,1,0,1,0,1,0,0.025033,0.044435,0.064166
3,3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,...,1,1,0,1,0,1,0,0.032262,0.036586,0.038931
4,4,4,100038,NaN,0,1,1,1,180000.0,625500.0,...,0,1,0,0,0,0,0,0.205365,0.141754,0.195575


In [25]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,1,0,0,0,0,0,0.042863,0.040068,0.049120
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,1,0,0,0,0,0,0.140563,0.095240,0.115729
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,1,0,1,0,1,0,0.025033,0.044435,0.064166
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,1,0,1,0,1,0,0.032262,0.036586,0.038931
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,1,0,0,0,0,0,0.205365,0.141754,0.195575


In [26]:
### level_0を削除
app_train.drop('level_0', axis=1, inplace=True)
app_test.drop('level_0', axis=1, inplace=True)
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,1,0,0,0,0,0,0.042863,0.040068,0.049120
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,1,1,0,0,0,0,0,0.140563,0.095240,0.115729
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,1,0,1,0,0.025033,0.044435,0.064166
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,1,0,1,0,1,0,0.032262,0.036586,0.038931
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,1,0,0,0,0,0,0.205365,0.141754,0.195575


In [27]:
app_train = FeatureEngineering2.reduce_mem_usage(app_train)
app_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1,1,0,0,0,0,0,0.284424,0.260986,0.223633
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,1,1,0,0,0,0,0,0.007950,0.012169,0.024139
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,1,1,0,0,0,0,0,0.013649,0.039948,0.035309
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0,1,0,1,0,0,0,0.033356,0.030777,0.031128
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,1,1,0,0,0,0,0,0.071838,0.035095,0.035156


In [28]:
app_test = FeatureEngineering2.reduce_mem_usage(app_test)
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,1,0,0,0,0,0,0.042877,0.040070,0.049133
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,1,1,0,0,0,0,0,0.140503,0.095215,0.115723
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,1,0,1,0,0.025040,0.044434,0.064148
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,1,0,1,0,1,0,0.032257,0.036591,0.038940
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,1,0,0,0,0,0,0.205322,0.141724,0.195557


### Xgbのmodelを生成

In [29]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [30]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [31]:
# infをnanへ置換
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_test = app_test.replace([np.inf, -np.inf], np.nan)
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,1,0,0,0,0,0,0.042877,0.040070,0.049133
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,1,1,0,0,0,0,0,0.140503,0.095215,0.115723
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,1,0,1,0,0.025040,0.044434,0.064148
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,1,0,1,0,1,0,0.032257,0.036591,0.038940
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,1,0,0,0,0,0,0.205322,0.141724,0.195557


In [32]:
### 処理対象の列名を配列化
encoding_columns = list(app_test.select_dtypes(include='number').columns)
    
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [33]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train, app_test, num_cols)
        
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,100001,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,...,0.579736,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,-0.414551,-0.419189,-0.328125
1,100005,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,...,0.579736,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,0.692383,0.158813,0.364990
2,100013,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,0.247943,...,0.579736,0.245284,0.0,1.685055,-0.150513,2.007802,0.0,-0.616699,-0.373535,-0.171875
3,100028,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,2.805819,...,0.579736,0.245284,0.0,1.685055,-0.150513,2.007802,0.0,-0.535156,-0.455566,-0.434326
4,100038,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,0.235763,...,-1.724923,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,1.426758,0.646484,1.195312


In [34]:
### indexを削除
app_train = app_train.reset_index()
app_test = app_test.reset_index()
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,0,100001,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,...,0.579736,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,-0.414551,-0.419189,-0.328125
1,1,100005,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,...,0.579736,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,0.692383,0.158813,0.364990
2,2,100013,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,...,0.579736,0.245284,0.0,1.685055,-0.150513,2.007802,0.0,-0.616699,-0.373535,-0.171875
3,3,100028,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,...,0.579736,0.245284,0.0,1.685055,-0.150513,2.007802,0.0,-0.535156,-0.455566,-0.434326
4,4,100038,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,...,-1.724923,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,1.426758,0.646484,1.195312


In [35]:
### training Xgb
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1a = Model1Xgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, x_train, y_train, x_test)

[11:04:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "num_round", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.78494	eval-auc:0.77951
[1]	train-auc:0.79232	eval-auc:0.78695
[2]	train-auc:0.79269	eval-auc:0.78719
[3]	train-auc:0.79372	eval-auc:0.78813
[4]	train-auc:0.79525	eval-auc:0.79113
[5]	train-auc:0.79517	eval-auc:0.79109
[6]	train-auc:0.79574	eval-auc:0.79140
[7]	train-auc:0.79584	eval-auc:0.79157
[8]	train-auc:0.79650	eval-auc:0.79128
[9]	train-auc:0.79662	eval-auc:0.79131
[10]	train-auc:0.79640	eval-auc:0.79107
[11]	train-auc:0.79655	eval-auc:0.79092
[12]	train-auc:0.79700	eval-auc:0.79164
[13]	train-auc:0.79689	eval-auc:0.79150
[14

[192]	train-auc:0.81539	eval-auc:0.79278
[193]	train-auc:0.81553	eval-auc:0.79278
[194]	train-auc:0.81564	eval-auc:0.79277
[195]	train-auc:0.81574	eval-auc:0.79276
[196]	train-auc:0.81586	eval-auc:0.79278
[197]	train-auc:0.81601	eval-auc:0.79275
[198]	train-auc:0.81622	eval-auc:0.79277
[199]	train-auc:0.81634	eval-auc:0.79277
[200]	train-auc:0.81646	eval-auc:0.79277
[201]	train-auc:0.81659	eval-auc:0.79279
[202]	train-auc:0.81669	eval-auc:0.79280
[203]	train-auc:0.81680	eval-auc:0.79281
[204]	train-auc:0.81698	eval-auc:0.79280
[205]	train-auc:0.81707	eval-auc:0.79281
[206]	train-auc:0.81720	eval-auc:0.79280
[207]	train-auc:0.81733	eval-auc:0.79282
[208]	train-auc:0.81755	eval-auc:0.79277
[209]	train-auc:0.81765	eval-auc:0.79276
[210]	train-auc:0.81782	eval-auc:0.79277
[211]	train-auc:0.81799	eval-auc:0.79274
[212]	train-auc:0.81811	eval-auc:0.79273
[213]	train-auc:0.81829	eval-auc:0.79272
[214]	train-auc:0.81844	eval-auc:0.79274
[215]	train-auc:0.81856	eval-auc:0.79274
[216]	train-auc:

[392]	train-auc:0.85266	eval-auc:0.79147
[393]	train-auc:0.85285	eval-auc:0.79149
[394]	train-auc:0.85305	eval-auc:0.79145
[395]	train-auc:0.85318	eval-auc:0.79143
[396]	train-auc:0.85338	eval-auc:0.79142
[397]	train-auc:0.85360	eval-auc:0.79140
[398]	train-auc:0.85382	eval-auc:0.79142
[399]	train-auc:0.85410	eval-auc:0.79140
[400]	train-auc:0.85431	eval-auc:0.79139
[401]	train-auc:0.85452	eval-auc:0.79139
[402]	train-auc:0.85468	eval-auc:0.79140
[403]	train-auc:0.85484	eval-auc:0.79141
[404]	train-auc:0.85501	eval-auc:0.79141
[405]	train-auc:0.85519	eval-auc:0.79140
[406]	train-auc:0.85534	eval-auc:0.79140
[407]	train-auc:0.85556	eval-auc:0.79142
[408]	train-auc:0.85578	eval-auc:0.79144
[409]	train-auc:0.85592	eval-auc:0.79145
[410]	train-auc:0.85611	eval-auc:0.79144
[411]	train-auc:0.85624	eval-auc:0.79143
[412]	train-auc:0.85639	eval-auc:0.79143
[413]	train-auc:0.85651	eval-auc:0.79141
[414]	train-auc:0.85667	eval-auc:0.79142
[415]	train-auc:0.85683	eval-auc:0.79142
[416]	train-auc:

[592]	train-auc:0.88492	eval-auc:0.79038
[593]	train-auc:0.88502	eval-auc:0.79036
[594]	train-auc:0.88525	eval-auc:0.79035
[595]	train-auc:0.88534	eval-auc:0.79035
[596]	train-auc:0.88548	eval-auc:0.79034
[597]	train-auc:0.88556	eval-auc:0.79034
[598]	train-auc:0.88561	eval-auc:0.79032
[599]	train-auc:0.88577	eval-auc:0.79031
[600]	train-auc:0.88588	eval-auc:0.79031
[601]	train-auc:0.88597	eval-auc:0.79029
[602]	train-auc:0.88618	eval-auc:0.79027
[603]	train-auc:0.88631	eval-auc:0.79025
[604]	train-auc:0.88647	eval-auc:0.79024
[605]	train-auc:0.88664	eval-auc:0.79022
[606]	train-auc:0.88674	eval-auc:0.79021
[607]	train-auc:0.88695	eval-auc:0.79021
[608]	train-auc:0.88710	eval-auc:0.79020
[609]	train-auc:0.88722	eval-auc:0.79021
[610]	train-auc:0.88734	eval-auc:0.79020
[611]	train-auc:0.88739	eval-auc:0.79020
[612]	train-auc:0.88764	eval-auc:0.79017
[613]	train-auc:0.88773	eval-auc:0.79016
[614]	train-auc:0.88787	eval-auc:0.79015
[615]	train-auc:0.88793	eval-auc:0.79015
[616]	train-auc:

[792]	train-auc:0.91170	eval-auc:0.78905
[793]	train-auc:0.91188	eval-auc:0.78906
[794]	train-auc:0.91195	eval-auc:0.78906
[795]	train-auc:0.91209	eval-auc:0.78907
[796]	train-auc:0.91218	eval-auc:0.78907
[797]	train-auc:0.91240	eval-auc:0.78906
[798]	train-auc:0.91254	eval-auc:0.78905
[799]	train-auc:0.91265	eval-auc:0.78905
[800]	train-auc:0.91278	eval-auc:0.78904
[801]	train-auc:0.91292	eval-auc:0.78903
[802]	train-auc:0.91301	eval-auc:0.78903
[803]	train-auc:0.91322	eval-auc:0.78902
[804]	train-auc:0.91332	eval-auc:0.78900
[805]	train-auc:0.91338	eval-auc:0.78900
[806]	train-auc:0.91348	eval-auc:0.78900
[807]	train-auc:0.91363	eval-auc:0.78898
[808]	train-auc:0.91372	eval-auc:0.78898
[809]	train-auc:0.91385	eval-auc:0.78897
[810]	train-auc:0.91398	eval-auc:0.78897
[811]	train-auc:0.91411	eval-auc:0.78897
[812]	train-auc:0.91421	eval-auc:0.78895
[813]	train-auc:0.91434	eval-auc:0.78895
[814]	train-auc:0.91443	eval-auc:0.78893
[815]	train-auc:0.91461	eval-auc:0.78892
[816]	train-auc:

...

[162]	train-auc:0.81145	eval-auc:0.79057
[163]	train-auc:0.81154	eval-auc:0.79058
[164]	train-auc:0.81166	eval-auc:0.79059
[165]	train-auc:0.81183	eval-auc:0.79057
[166]	train-auc:0.81203	eval-auc:0.79056
[167]	train-auc:0.81214	eval-auc:0.79057
[168]	train-auc:0.81229	eval-auc:0.79053
[169]	train-auc:0.81244	eval-auc:0.79053
[170]	train-auc:0.81259	eval-auc:0.79054
[171]	train-auc:0.81280	eval-auc:0.79049
[172]	train-auc:0.81295	eval-auc:0.79050
[173]	train-auc:0.81306	eval-auc:0.79051
[174]	train-auc:0.81324	eval-auc:0.79048
[175]	train-auc:0.81335	eval-auc:0.79050
[176]	train-auc:0.81352	eval-auc:0.79051
[177]	train-auc:0.81368	eval-auc:0.79053
[178]	train-auc:0.81380	eval-auc:0.79052
[179]	train-auc:0.81387	eval-auc:0.79052
[180]	train-auc:0.81403	eval-auc:0.79053
[181]	train-auc:0.81415	eval-auc:0.79053
[182]	train-auc:0.81433	eval-auc:0.79050
[183]	train-auc:0.81447	eval-auc:0.79049
[184]	train-auc:0.81456	eval-auc:0.79050
[185]	train-auc:0.81487	eval-auc:0.79051
[186]	train-auc:

[362]	train-auc:0.84745	eval-auc:0.78979
[363]	train-auc:0.84758	eval-auc:0.78979
[364]	train-auc:0.84776	eval-auc:0.78980
[365]	train-auc:0.84803	eval-auc:0.78979
[366]	train-auc:0.84820	eval-auc:0.78979
[367]	train-auc:0.84843	eval-auc:0.78977
[368]	train-auc:0.84864	eval-auc:0.78978
[369]	train-auc:0.84884	eval-auc:0.78978
[370]	train-auc:0.84904	eval-auc:0.78976
[371]	train-auc:0.84926	eval-auc:0.78975
[372]	train-auc:0.84948	eval-auc:0.78972
[373]	train-auc:0.84970	eval-auc:0.78972
[374]	train-auc:0.84995	eval-auc:0.78973
[375]	train-auc:0.85022	eval-auc:0.78972
[376]	train-auc:0.85040	eval-auc:0.78972
[377]	train-auc:0.85056	eval-auc:0.78971
[378]	train-auc:0.85076	eval-auc:0.78969
[379]	train-auc:0.85100	eval-auc:0.78968
[380]	train-auc:0.85123	eval-auc:0.78968
[381]	train-auc:0.85137	eval-auc:0.78966
[382]	train-auc:0.85153	eval-auc:0.78967
[383]	train-auc:0.85175	eval-auc:0.78965
[384]	train-auc:0.85191	eval-auc:0.78963
[385]	train-auc:0.85216	eval-auc:0.78961
[386]	train-auc:

[562]	train-auc:0.88255	eval-auc:0.78897
[563]	train-auc:0.88268	eval-auc:0.78896
[564]	train-auc:0.88283	eval-auc:0.78896
[565]	train-auc:0.88299	eval-auc:0.78896
[566]	train-auc:0.88309	eval-auc:0.78895
[567]	train-auc:0.88323	eval-auc:0.78895
[568]	train-auc:0.88346	eval-auc:0.78895
[569]	train-auc:0.88358	eval-auc:0.78894
[570]	train-auc:0.88371	eval-auc:0.78893
[571]	train-auc:0.88383	eval-auc:0.78892
[572]	train-auc:0.88395	eval-auc:0.78892
[573]	train-auc:0.88403	eval-auc:0.78891
[574]	train-auc:0.88417	eval-auc:0.78891
[575]	train-auc:0.88431	eval-auc:0.78890
[576]	train-auc:0.88439	eval-auc:0.78889
[577]	train-auc:0.88450	eval-auc:0.78890
[578]	train-auc:0.88467	eval-auc:0.78890
[579]	train-auc:0.88478	eval-auc:0.78890
[580]	train-auc:0.88493	eval-auc:0.78890
[581]	train-auc:0.88506	eval-auc:0.78889
[582]	train-auc:0.88518	eval-auc:0.78887
[583]	train-auc:0.88534	eval-auc:0.78887
[584]	train-auc:0.88553	eval-auc:0.78886
[585]	train-auc:0.88568	eval-auc:0.78886
[586]	train-auc:

[762]	train-auc:0.90761	eval-auc:0.78796
[763]	train-auc:0.90772	eval-auc:0.78795
[764]	train-auc:0.90780	eval-auc:0.78795
[765]	train-auc:0.90797	eval-auc:0.78793
[766]	train-auc:0.90806	eval-auc:0.78794
[767]	train-auc:0.90819	eval-auc:0.78794
[768]	train-auc:0.90836	eval-auc:0.78793
[769]	train-auc:0.90848	eval-auc:0.78792
[770]	train-auc:0.90864	eval-auc:0.78791
[771]	train-auc:0.90881	eval-auc:0.78791
[772]	train-auc:0.90899	eval-auc:0.78789
[773]	train-auc:0.90911	eval-auc:0.78788
[774]	train-auc:0.90929	eval-auc:0.78786
[775]	train-auc:0.90938	eval-auc:0.78786
[776]	train-auc:0.90951	eval-auc:0.78784
[777]	train-auc:0.90968	eval-auc:0.78784
[778]	train-auc:0.90989	eval-auc:0.78782
[779]	train-auc:0.91001	eval-auc:0.78784
[780]	train-auc:0.91012	eval-auc:0.78782
[781]	train-auc:0.91028	eval-auc:0.78781
[782]	train-auc:0.91043	eval-auc:0.78781
[783]	train-auc:0.91055	eval-auc:0.78781
[784]	train-auc:0.91066	eval-auc:0.78780
[785]	train-auc:0.91072	eval-auc:0.78780
[786]	train-auc:

[962]	train-auc:0.93226	eval-auc:0.78695
[963]	train-auc:0.93244	eval-auc:0.78695
[964]	train-auc:0.93258	eval-auc:0.78695
[965]	train-auc:0.93269	eval-auc:0.78695
[966]	train-auc:0.93282	eval-auc:0.78696
[967]	train-auc:0.93289	eval-auc:0.78697
[968]	train-auc:0.93304	eval-auc:0.78696
[969]	train-auc:0.93311	eval-auc:0.78696
[970]	train-auc:0.93321	eval-auc:0.78697
[971]	train-auc:0.93339	eval-auc:0.78697
[972]	train-auc:0.93348	eval-auc:0.78698
[973]	train-auc:0.93361	eval-auc:0.78698
[974]	train-auc:0.93374	eval-auc:0.78695
[975]	train-auc:0.93391	eval-auc:0.78692
[976]	train-auc:0.93404	eval-auc:0.78692
[977]	train-auc:0.93417	eval-auc:0.78692
[978]	train-auc:0.93425	eval-auc:0.78692
[979]	train-auc:0.93443	eval-auc:0.78689
[980]	train-auc:0.93454	eval-auc:0.78687
[981]	train-auc:0.93458	eval-auc:0.78686
[982]	train-auc:0.93470	eval-auc:0.78685
[983]	train-auc:0.93480	eval-auc:0.78684
[984]	train-auc:0.93490	eval-auc:0.78685
[985]	train-auc:0.93503	eval-auc:0.78684
[986]	train-auc:

In [36]:
### 1層目のモデルの評価
#print(f'logloss: {log_loss(y_train, pred_train_1a, eps=1e-7):.4f}')
#print(f'logloss: {log_loss(y_train, pred_train_1b, eps=1e-7):.4f}')

In [37]:
np.array(pred_train_1a)

array([0.19879371, 0.01264763, 0.03306   , ..., 0.03171205, 0.10083564,
       0.07386818], dtype=float32)

In [38]:
pred_1a = []
for a in np.array(pred_train_1a):
    pred_1a.append(a)

len(pred_1a)

307506

In [39]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set2x': pred_1a})
print(train_x_2.shape)
train_x_2.head()

(307506, 2)


,SK_ID_CURR,set2x
0,100002,0.198794
1,100003,0.012648
2,100004,0.033060
3,100006,0.025442
4,100007,0.039186


In [40]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/train_set2xgb.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [41]:
pred_1a = []
for a in np.array(pred_test_1a):
    pred_1a.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1a)

48744

In [42]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set2x': pred_1a})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,set2x
0,100001,0.035013
1,100005,0.103532
2,100013,0.048415
3,100028,0.032249
4,100038,0.150442


In [43]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/test_set2xgb.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [44]:
#del df
#del x_train
#del y_train
#del x_test
#del app_train
#del app_test
#del model_1a
#gc.collect()